In [477]:
from utils import *
from preprocessing import *
from models_date import *
from models_sex_prediction import *


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [457]:
X_train, y_train, X_test = read_train_test(sentence_per_row_mode=True)

Reading training data ...
Reading training data : Done
Reading testing data ...
Reading testing data : Done


In [458]:
X_train["text_processed"] = X_train.text.apply(process_text, args=(False,))

In [459]:
X_train.head()

,filename,text,text_processed
ID,,,
0,Agen_100515.txt,"[Le : 12/11/2019, , , Cour d’appel d’Agen , , ...","[: 12/11/2019, , , cour appel agen, , chambre ..."
1,Agen_1100752.txt,"[Le : 12/11/2019, , , Cour d’appel d’Agen , , ...","[: 12/11/2019, , , cour appel agen, , chambre ..."
2,Agen_1613.txt,"[Le : 12/11/2019, , , Cour d’appel d’Agen , , ...","[: 12/11/2019, , , cour appel agen, , audience..."
3,Agen_2118.txt,"[Le : 12/11/2019, , , Cour d’appel d’Agen , , ...","[: 12/11/2019, , , cour appel agen, , audience..."
4,Agen_21229.txt,"[Le : 12/11/2019, , , Cour d’appel d’Agen , , ...","[: 12/11/2019, , , cour appel agen, , audience..."


***

# Extraction du contexte autour de la date (utilisant la target )

In [48]:
# import itertools
# import igraph
# import copy

# def terms_to_graph(terms, window_size):
#     '''This function returns a directed, weighted igraph from lists of list of terms (the tokens from the pre-processed text)
#     e.g., ['quick','brown','fox']
#     Edges are weighted based on term co-occurence within a sliding window of fixed size 'w'
#     '''
    
#     from_to = {}

#     w = min(window_size, len(terms))
#     # create initial complete graph (first w terms)
#     terms_temp = terms[0:w]
#     indexes = list(itertools.combinations(range(w), r=2))

#     new_edges = []

#     for my_tuple in indexes:
#         new_edges.append(tuple([terms_temp[i] for i in my_tuple]))
#     for new_edge in new_edges:
#         if new_edge in from_to:
#             from_to[new_edge] += 1
#         else:
#             from_to[new_edge] = 1

#     # then iterate over the remaining terms
#     for i in range(w, len(terms)):
#         # term to consider
#         considered_term = terms[i]
#         # all terms within sliding window
#         terms_temp = terms[(i - w + 1):(i + 1)]

#         # edges to try
#         candidate_edges = []
#         for p in range(w - 1):
#             candidate_edges.append((terms_temp[p], considered_term))

#         for try_edge in candidate_edges:

#             # if not self-edge
#             if try_edge[1] != try_edge[0]:

#                 # if edge has already been seen, update its weight
#                 if try_edge in from_to:
#                     from_to[try_edge] += 1

#                 # if edge has never been seen, create it and assign it a unit weight
#                 else:
#                     from_to[try_edge] = 1

#     # create empty graph
#     g = igraph.Graph(directed=True)

#     # add vertices
#     g.add_vertices(sorted(set(terms)))

#     # add edges, direction is preserved since the graph is directed
#     g.add_edges(list(from_to.keys()))

#     # set edge and vertice weights
#     g.es['weight'] = list(from_to.values()) # based on co-occurence within sliding window
#     g.vs['weight'] = g.strength(weights=list(from_to.values())) # weighted degree

#     return (g)

# def core_dec(g,weighted):
#     '''(un)weighted k-core decomposition'''
#     # work on clone of g to preserve g 
#     gg = copy.deepcopy(g)
#     if not weighted:
#         gg.vs['weight'] = gg.strength() # overwrite the 'weight' vertex attribute with the unweighted degrees
#     # initialize dictionary that will contain the core numbers
#     cores_g = dict(zip(gg.vs['name'],[0]*len(gg.vs)))
    
#     while len(gg.vs) > 0:
#         # find index of lowest degree vertex
#         min_degree = min(gg.vs['weight'])
#         index_top = gg.vs['weight'].index(min_degree)
#         name_top = gg.vs[index_top]['name']
#         # get names of its neighbors
#         neighbors = gg.vs[gg.neighbors(index_top)]['name']
#         # exclude self-edges
#         neighbors = [elt for elt in neighbors if elt!=name_top]
#         # set core number of lowest degree vertex as its degree
#         cores_g[name_top] = min_degree
#         # delete top vertex and its incident edges
#         gg.delete_vertices(index_top)
        
#         if neighbors:
#             if weighted: 
#                 new_degrees = gg.strength(weights=gg.es['weight'])
#             else:
#                 new_degrees = gg.strength()
#             # iterate over neighbors of top element
#             for neigh in neighbors:
#                 index_n = gg.vs['name'].index(neigh)
#                 gg.vs[index_n]['weight'] = max(min_degree,new_degrees[index_n])  
        
#     return(cores_g)


In [12]:
# context_to_analyse = extract_X_sentences_before_after(X_train.text_processed[20], y_train.date_accident[20], X=1)


# context_to_analyse = extract_X_sentences_around_all_dates(X_train.text_processed[20], 
#                                                           terms_discarding_the_date, 
#                                                           use_date_forcing=True,
#                                                           date_to_look_for=y_train.date_accident[20])
# context_to_analyse


# Contexte autour de la date d':

- Renvoie `None`  si : 
    - La date dans y n'a pas été trouvée sous le format jour (nombre ou "1er") mois (en letter) année : 11 decembre 2007 | 1er avril 2006
    - La date est n.c.

## Accident

In [460]:
L = []
for i in range(X_train.shape[0]) :
    context_to_analyse = extract_X_sentences_around_all_dates(X_train.text_processed.iloc[i], 
                                                              terms_discarding_the_date, 
                                                              use_date_forcing=True,
                                                              date_to_look_for=y_train.date_accident.iloc[i])
    if (context_to_analyse == []) or context_to_analyse == "Date NC" :
        L.append(None)

    else :
        L.append(context_to_analyse[0][0]) # je ne prends que le premier contexte pour commencer
#         L.append(" ".join(np.array(context_to_analyse)[:,0]))
X_train["context_around_accident"] = L

## Consolidation

In [461]:
L = []
for i in range(X_train.shape[0]) :
    context_to_analyse = extract_X_sentences_around_all_dates(X_train.text_processed.iloc[i], 
                                                              terms_discarding_the_date, 
                                                              use_date_forcing=True,
                                                              date_to_look_for=y_train.date_consolidation.iloc[i])
    if (context_to_analyse == []) or context_to_analyse == "Date NC" :
        L.append(None)
    else :
        L.append(context_to_analyse[0][0]) # je ne prends que le premier contexte pour commencer
#         L.append(" ".join(np.array(context_to_analyse)[:,0])) # concatener tout les contextes
X_train["context_around_consolidation"] = L

## Spacy word vectors

In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [462]:
X_tr, X_te = train_test_split(X_train, random_state=50)

<IPython.core.display.Javascript object>

To asses the performance of our model that predicts if the context around a date is talking about `date d'accident` or `date de consolidation` :

- Take split X_train using train_test_split into X_tr and X_te
- Prepare data for model fitting :
    - For date d'accident : 
        - remove the dates from the accident contexts of X_tr
        - remove the punctuation from the accident contexts of X_tr
        - remove the stopwords from the accident contexts of X_tr  
        - use Spacy avg vectorizer or tfidf
    - Same thing for date de conso
    - concatenate the two 
    - fit two models : one to predict accident contexts and the other the consolidation contexts
- On X_te :
    - On each text apply the function extract `X_sentences_around_all_dates`
    - for each sentence apply the Spacy vectorizer and predict using the models
    - predict the date of which the context gave the higher probability score.

Results : 
- accident : 35% accuracy with tf idf approach on context extracted with X=1 and SVM classifier
- conso : 24% accuracy with spacy word vectors approach on context extracted with X=1 and SVM classifier

In [346]:
# train_accident_context_avg_WV = np.array([
#     nlp(sen).vector for sen in X_tr.context_around_accident_processed if sen is not None ])

# train_consolidation_context_avg_WV = np.array([
#     nlp(sen).vector for sen in X_tr.context_around_consolidation_processed if sen is not None ])

train_accident_context_avg_WV = np.array([
    nlp(sen).vector for sen in X_tr.context_around_accident if sen is not None ])

train_consolidation_context_avg_WV = np.array([
    nlp(sen).vector for sen in X_tr.context_around_consolidation if sen is not None ])


 
is_accident = [1 for _ in range(train_accident_context_avg_WV.shape[0])]
is_not_accident = [0 for _ in range(train_consolidation_context_avg_WV.shape[0])]
data = np.concatenate((train_accident_context_avg_WV, train_consolidation_context_avg_WV))
target = np.concatenate((is_accident, is_not_accident))

# clf_accident = LogisticRegression().fit(data, target)
clf_accident = SVC(gamma='auto', probability=True).fit(data, target)

###############################################

is_conso = [1 for _ in range(train_consolidation_context_avg_WV.shape[0])]
is_not_conso = [0 for _ in range(train_accident_context_avg_WV.shape[0])]
data = np.concatenate((train_consolidation_context_avg_WV, train_accident_context_avg_WV))
target = np.concatenate((is_conso, is_not_conso))

# clf_conso = LogisticRegression().fit(data, target)
clf_conso = SVC(gamma='auto', probability=True).fit(data, target)

In [38]:
def Word_vector_SPACY(text) :
    """For a given context : gives the average spacy word vectors of the context
    used in df.apply() of a 'context_around_...' column of a dataframe
    """
    if text is None :
        return None
    else :
        return nlp(text).vector 

In [463]:
train_accident_context_avg_WV = X_tr.context_around_accident.apply(Word_vector_SPACY)
train_consolidation_context_avg_WV = X_tr.context_around_consolidation.apply(Word_vector_SPACY)

In [464]:
data_is_accident_context = np.matrix(train_accident_context_avg_WV[train_accident_context_avg_WV.isnull() == False].to_list())
data_is_consolidation_context = np.matrix(train_consolidation_context_avg_WV[train_consolidation_context_avg_WV.isnull() == False].to_list())

In [465]:
data = np.concatenate((data_is_accident_context, data_is_consolidation_context))
target_accident = np.concatenate(([1 for _ in range(data_is_accident_context.shape[0])],
                                 [0 for _ in range(data_is_consolidation_context.shape[0])]))

target_consolidation = np.concatenate(([0 for _ in range(data_is_accident_context.shape[0])],
                                        [1 for _ in range(data_is_consolidation_context.shape[0])]))

In [466]:
# train_accident_context_avg_WV[train_accident_context_avg_WV.isnull() == False].index
# train_consolidation_context_avg_WV[train_consolidation_context_avg_WV.isnull() == False].index

In [467]:
clf_accident = SVC(gamma='auto', probability=True).fit(data, target_accident)
clf_consolidation = SVC(gamma='auto', probability=True).fit(data, target_consolidation)

In [123]:
def date_prediction_classifier_Word_vectors(text, clf, threshold=0.7):
    
    probas_ = []
    sentences_to_test = extract_X_sentences_around_all_dates(text, terms_discarding_the_date)
    for tup in sentences_to_test :
        processed_sentence = tup[0]
        WV_processed_sentence = nlp(processed_sentence).vector
        probas_.append(clf.predict_proba([WV_processed_sentence]))

    probas_ = np.array(probas_)
    if probas_.shape[0] == 1 :
        probas_ = probas_[0]
    else :
        probas_ = np.squeeze(probas_)[:,1]
    
    if probas_.max() >= threshold :
        return letter_date_to_submission_date(sentences_to_test[np.argmax(probas_)][1])
    else :
        return "n.c."

In [478]:
y_pred_accident = X_te.text_processed.apply(date_prediction_classifier_Word_vectors, args=(clf_accident,))
y_pred_consolidation = X_te.text_processed.apply(date_prediction_classifier_Word_vectors, args=(clf_consolidation,))

In [479]:
y_true_accident = y_train.date_accident.loc[X_te.index]
y_true_consolidation = y_train.date_consolidation.loc[X_te.index]

In [480]:
(y_true_accident == y_pred_accident).mean(), (y_pred_consolidation == y_true_consolidation).mean()

(0.49740932642487046, 0.41968911917098445)

## Tf idf approach

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [516]:
data_is_accident_corpus = X_tr.context_around_accident[X_tr.context_around_accident.isnull() == False].to_list()
data_is_consolidation_corpus = X_tr.context_around_consolidation[X_tr.context_around_consolidation.isnull() == False].to_list()

In [517]:

data = np.concatenate((data_is_accident_corpus, data_is_consolidation_corpus))

vectorizer = TfidfVectorizer()
data_tfidf = vectorizer.fit_transform(data)

target_accident = np.concatenate(([1 for _ in range(len(data_is_accident_corpus))],
                                 [0 for _ in range(len(data_is_consolidation_corpus))]))

target_consolidation = np.concatenate(([0 for _ in range(len(data_is_accident_corpus))],
                                        [1 for _ in range(len(data_is_consolidation_corpus))]))

In [518]:

clf_accident = SVC(gamma='auto', probability=True).fit(data_tfidf, target_accident)
clf_consolidation = SVC(gamma='auto', probability=True).fit(data_tfidf, target_consolidation)

In [249]:
def date_prediction_classifier_tfidf(text, clf, vectorizer, threshold=0.7):
    
    probas_ = []
    sentences_to_test = extract_X_sentences_around_all_dates(text, terms_discarding_the_date)
    for tup in sentences_to_test :
        processed_sentence = tup[0]
        tfidf_encoded_processed_sentence = vectorizer.transform([processed_sentence])
        probas_.append(clf.predict_proba(tfidf_encoded_processed_sentence))

    probas_ = np.array(probas_)
    if probas_.shape[0] == 1 :
        probas_ = probas_[0][0]
        if probas_.max() >= threshold :
            return letter_date_to_submission_date(sentences_to_test[0][1])
        else :
            return "n.c."
        
    else :
        probas_ = np.squeeze(probas_)[:,1]

        if probas_.max() >= threshold :
            return letter_date_to_submission_date(sentences_to_test[np.argmax(probas_)][1])
        else :
            return "n.c."

In [519]:

y_pred_accident = X_te.text_processed.apply(date_prediction_classifier_tfidf, args=(clf_accident, vectorizer,))
y_pred_consolidation = X_te.text_processed.apply(date_prediction_classifier_tfidf, args=(clf_consolidation, vectorizer,))

In [520]:
y_true_accident = y_train.date_accident.loc[X_te.index]
y_true_consolidation = y_train.date_consolidation.loc[X_te.index]

In [521]:
(y_true_accident == y_pred_accident).mean(), (y_pred_consolidation == y_true_consolidation).mean()

(0.5906735751295337, 0.5025906735751295)

# Three classes approach 

In [402]:
terms_discarding_the_date = ["loi",
                             "jugement",
                             "audience",
                             "publique",
#                              "juge",
                             "tribubal",
                             "decision",
                             "greffe",
                             "conclusion",
                             "ordonnance",
                             ]
def extract_X_sentences_around_all_dates_other_dates(text, terms_discarding_the_date= terms_discarding_the_date, 
                                                     X=1):
    """
    The text in structured format ( like in the column text of X_train)
    - Looks for all the dates in the text
    - for each date, extract X sentences  before and after this date (sentence meaning rows of the original doc)
    - if a row has multiple dates, separate each date in a row (the dates will have the same context) ==> PROBLEM7
    - remove the contexts taht contain the words in terms_discarding_the_date (meaning that those are probably other dates) ==> Build a classifier 3 classes after
    - Return a list of tuples (context in lower + no stopwords + clean from punct , date)
    - this context can be passed to Spacy avg vectorizer to get the avg Word embedding of the sentence
    """

    l = [re.findall("\d{1,2} [a-zéû]{3,9} \d{4}", STRING) for STRING in text]
    indexes = [i for i in range(len(l)) if len(l[i])!=0]
    ll = [(" ".join(text[i-X+1 : i+X]), l[i]) for i in indexes]

    lll = []
    for i in range(len(ll)) :
        if len([word for word in terms_discarding_the_date if word in ll[i][0]]) > 0 :
            if len(ll[i][1]) == 1 :
                date = ll[i][1][0]
                context_date_removed = ll[i][0].replace(date, '')
                lll.append((context_date_removed, date))
                
#     return " ".join([lll[ind][0] for ind in range(len(lll))])

    ind = np.random.choice(range(len(lll))) # je ne prends qu'un contexte aleatoirement pour commencer
    return lll[ind][0]

## Autre dates

In [554]:
L = []
for i in range(X_train.shape[0]) :
    context_to_analyse = extract_X_sentences_around_all_dates_other_dates(X_train.text_processed.iloc[i])
    
    context_to_analyse_cleaned = ""
    for element in context_to_analyse :
        if not element.isalnum():
            if element == " " :
                context_to_analyse_cleaned += element 
        else :
            context_to_analyse_cleaned += element 
    L.append(context_to_analyse_cleaned)
    
X_train["context_around_autre_date_random"] = L

In [555]:
X_tr, X_te = train_test_split(X_train, random_state=50)

<IPython.core.display.Javascript object>

## Word vectors approach

In [489]:
train_accident_context_avg_WV = X_tr.context_around_accident.apply(Word_vector_SPACY)
train_consolidation_context_avg_WV = X_tr.context_around_consolidation.apply(Word_vector_SPACY)
train_other_context_avg_WV = X_tr.context_around_autre_date_random.apply(Word_vector_SPACY)

In [490]:
data_accident_context = np.matrix(train_accident_context_avg_WV[train_accident_context_avg_WV.isnull() == False].to_list())
data_consolidation_context = np.matrix(train_consolidation_context_avg_WV[train_consolidation_context_avg_WV.isnull() == False].to_list())
data_other_context = np.matrix(train_other_context_avg_WV[train_other_context_avg_WV.isnull() == False].to_list())

In [491]:

data = np.concatenate((data_accident_context, data_consolidation_context, data_other_context))

In [492]:

is_accident = [0 for _ in range(data_accident_context.shape[0])]
is_conso = [1 for _ in range(data_consolidation_context.shape[0])]
is_other = [2 for _ in range(data_other_context.shape[0])]

In [493]:
target = np.concatenate((is_accident, is_conso, is_other))

In [494]:
clf = SVC(gamma='auto', probability=True).fit(data, target)

In [280]:
def date_prediction_classifier_Word_vectors_multiclass(text, clf, threshold=0.35):
    
    probas_ = []
    sentences_to_test = extract_X_sentences_around_all_dates(text, terms_discarding_the_date)
    for tup in sentences_to_test :
        processed_sentence = tup[0]
        WV_processed_sentence = nlp(processed_sentence).vector
        probas_.append(clf.predict_proba([WV_processed_sentence]))

    probas_ = np.array(probas_)
    if probas_.shape[0] == 1 :
        probas_ = probas_[0][0]
        if np.argmax(probas_) == 0 :
            return [letter_date_to_submission_date(sentences_to_test[0][1]), "n.c."]
        elif np.argmax(probas_) == 1 :
            return ["n.c.", letter_date_to_submission_date(sentences_to_test[0][1])] 
        else :
            return ["n.c.", "n.c."]
    else :
        probas_ = np.squeeze(probas_)
        
        event = ["accident", "conso"]
        decisions = []
        for row in probas_ :
            if np.argmax(row) == 2 :
                decisions.append("other")
            elif max(row[:2])<= threshold :
                decisions.append("other")
            else :
                decisions.append(event[np.argmax(row[:2])])
        
        ind_accident = None
        ind_conso = None
        prev_prob_acc = 0
        prev_prob_conso = 0
        
        for i in range(probas_.shape[0]):
            prob_acc = probas_[i,0]
            if (decisions[i] == "accident") & (prob_acc >= prev_prob_acc) :
                ind_accident = i
                prev_prob_acc = prob_acc
                
            prob_conso = probas_[i,1]
            if (decisions[i] == "conso") & (prob_conso >= prev_prob_conso):
                ind_conso = i
                prev_prob_conso = prob_conso
                
        if ind_accident is None :
            date_accident = "n.c."
        else :
            date_accident = letter_date_to_submission_date(sentences_to_test[ind_accident][1])
            
        if ind_conso is None :
            date_conso = "n.c."
        else :
            date_conso = letter_date_to_submission_date(sentences_to_test[ind_conso][1])
        
        return [date_accident, date_conso]

In [495]:
y_pred = X_te.text_processed.apply(date_prediction_classifier_Word_vectors_multiclass, args=(clf,))


In [496]:
y_pred_accident = pd.DataFrame(y_pred.values.tolist(), index= y_pred.index)[0]
y_pred_consolidation = pd.DataFrame(y_pred.values.tolist(), index= y_pred.index)[1]

In [497]:
y_true_accident = y_train.date_accident.loc[X_te.index]
y_true_consolidation = y_train.date_consolidation.loc[X_te.index]

In [498]:
(y_true_accident == y_pred_accident).mean(), (y_pred_consolidation == y_true_consolidation).mean()

(0.5492227979274611, 0.38341968911917096)

## Tf idf approach

In [420]:
def date_prediction_classifier_tfidf_multiclass(text, clf, vectorizer, threshold=0.35):
    
    probas_ = []
    sentences_to_test = extract_X_sentences_around_all_dates(text, terms_discarding_the_date=[])
    for tup in sentences_to_test :
        processed_sentence = tup[0]
        tfidf_encoded_processed_sentence = vectorizer.transform([processed_sentence])
        probas_.append(clf.predict_proba(tfidf_encoded_processed_sentence))

    probas_ = np.array(probas_)
    if probas_.shape[0] == 1 :
        probas_ = probas_[0][0]
        if np.argmax(probas_) == 0 :
            return [letter_date_to_submission_date(sentences_to_test[0][1]), "n.c."]
        elif np.argmax(probas_) == 1 :
            return ["n.c.", letter_date_to_submission_date(sentences_to_test[0][1])] 
        else :
            return ["n.c.", "n.c."]
    else :
        probas_ = np.squeeze(probas_)
        
        event = ["accident", "conso"]
        decisions = []
        for row in probas_ :
            if np.argmax(row) == 2 :
                decisions.append("other")
            elif max(row[:2])<= threshold :
                decisions.append("other")
            else :
                decisions.append(event[np.argmax(row[:2])])
        
        ind_accident = None
        ind_conso = None
        prev_prob_acc = 0
        prev_prob_conso = 0
        
        for i in range(probas_.shape[0]):
            prob_acc = probas_[i,0]
            if (decisions[i] == "accident") & (prob_acc >= prev_prob_acc) :
                ind_accident = i
                prev_prob_acc = prob_acc
                
            prob_conso = probas_[i,1]
            if (decisions[i] == "conso") & (prob_conso >= prev_prob_conso):
                ind_conso = i
                prev_prob_conso = prob_conso
                
        if ind_accident is None :
            date_accident = "n.c."
        else :
            date_accident = letter_date_to_submission_date(sentences_to_test[ind_accident][1])
            
        if ind_conso is None :
            date_conso = "n.c."
        else :
            date_conso = letter_date_to_submission_date(sentences_to_test[ind_conso][1])
        
        return [date_accident, date_conso]

In [547]:
data_is_accident_corpus = X_tr.context_around_accident[X_tr.context_around_accident.isnull() == False].to_list()
data_is_consolidation_corpus = X_tr.context_around_consolidation[X_tr.context_around_consolidation.isnull() == False].to_list()
data_other_context = X_tr.context_around_autre_date_random[X_tr.context_around_autre_date_random.isnull() == False].to_list()
data = np.concatenate((data_is_accident_corpus, data_is_consolidation_corpus, data_other_context))

vectorizer = TfidfVectorizer()
data_tfidf = vectorizer.fit_transform(data)


In [556]:
is_accident = [0 for _ in range(len(data_is_accident_corpus))]
is_conso = [1 for _ in range(len(data_is_consolidation_corpus))]
is_other = [2 for _ in range(len(data_other_context))]
target = np.concatenate((is_accident, is_conso, is_other))

In [557]:
clf = SVC(gamma='auto', probability=True).fit(data_tfidf, target)

In [558]:
y_pred = X_te.text_processed.apply(date_prediction_classifier_tfidf_multiclass, args=(clf, vectorizer,))

In [559]:
y_pred_accident = pd.DataFrame(y_pred.values.tolist(), index= y_pred.index)[0]
y_pred_consolidation = pd.DataFrame(y_pred.values.tolist(), index= y_pred.index)[1]

In [560]:
y_true_accident = y_train.date_accident.loc[X_te.index]
y_true_consolidation = y_train.date_consolidation.loc[X_te.index]

In [561]:
(y_true_accident == y_pred_accident).mean(), (y_pred_consolidation == y_true_consolidation).mean()

(0.6528497409326425, 0.44041450777202074)

# Submission

## Submission 1 

- Naive sex predictor version 1 
- accident : tfidf 3 classes
- consolidation : tf idf 2 classes

In [511]:
sex_prediction = X_test.text.apply(naive_sex_classifier)

In [513]:
X_test["text_processed"] = X_test.text.apply(process_text, args=(False,))

In [571]:
data_is_accident_corpus = X_train.context_around_accident[X_train.context_around_accident.isnull() == False].to_list()
data_is_consolidation_corpus = X_train.context_around_consolidation[X_train.context_around_consolidation.isnull() == False].to_list()
data = np.concatenate((data_is_accident_corpus, data_is_consolidation_corpus))

vectorizer = TfidfVectorizer()
data_tfidf = vectorizer.fit_transform(data)


target_consolidation = np.concatenate(([0 for _ in range(len(data_is_accident_corpus))],
                                        [1 for _ in range(len(data_is_consolidation_corpus))]))
clf_consolidation = SVC(gamma='auto', probability=True).fit(data_tfidf, target_consolidation)
y_pred_consolidation = X_test.text_processed.apply(date_prediction_classifier_tfidf, args=(clf_consolidation, vectorizer,))

In [572]:
data_is_accident_corpus = X_train.context_around_accident[X_train.context_around_accident.isnull() == False].to_list()
data_is_consolidation_corpus = X_train.context_around_consolidation[X_train.context_around_consolidation.isnull() == False].to_list()
data_other_context = X_train.context_around_autre_date_random[X_train.context_around_autre_date_random.isnull() == False].to_list()
data = np.concatenate((data_is_accident_corpus, data_is_consolidation_corpus, data_other_context))

vectorizer = TfidfVectorizer()
data_tfidf = vectorizer.fit_transform(data)

is_accident = [0 for _ in range(len(data_is_accident_corpus))]
is_conso = [1 for _ in range(len(data_is_consolidation_corpus))]
is_other = [2 for _ in range(len(data_other_context))]
target = np.concatenate((is_accident, is_conso, is_other))
clf = SVC(gamma='auto', probability=True).fit(data_tfidf, target)
y_pred = X_test.text_processed.apply(date_prediction_classifier_tfidf_multiclass, args=(clf, vectorizer,))
y_pred_accident = pd.DataFrame(y_pred.values.tolist(), index= y_pred.index)[0]

In [598]:
submit = pd.DataFrame(index = y_pred_accident.index)
submit["sexe"] = sex_prediction
submit["date_accident"] = y_pred_accident
submit["date_consolidation"] = y_pred_consolidation
submit.index = submit.index.rename("filename")
submit

,sexe,date_accident,date_consolidation
filename,,,
770,homme,2014-10-07,n.c.
771,homme,1996-07-05,n.c.
772,femme,1989-05-11,n.c.
773,homme,2001-01-25,n.c.
774,homme,1999-05-28,2001-05-28
...,...,...,...
1022,homme,2011-05-23,2005-04-07
1023,femme,1968-05-20,1968-05-20
1024,femme,1948-05-13,2012-01-26


In [599]:
submit.to_csv("Sub1.csv")